<a href="https://colab.research.google.com/github/teploe-odealko/meteoinfo_parsing/blob/main/covid_%2B_graphics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Загрузка шаблонов

Шаблон табличной сводки по миру - файл xlsx

In [ ]:
from google.colab import files
world_input_filename = files.upload()
world_input_filename = next(iter(world_input_filename))

Saving МИР21_27-07-2021_28-07-2021.xlsx to МИР21_27-07-2021_28-07-2021.xlsx


Табличня сводка по россии - файл xlsx

In [ ]:
russia_input_filename = files.upload()
russia_input_filename = next(iter(russia_input_filename))

Презентация с графиками

In [ ]:
prs_filename = files.upload()
prs_filename = next(iter(prs_filename))

# Install & import

In [ ]:
import time
start_time = time.time()

In [ ]:
import os
from datetime import datetime
today = datetime.today().strftime("%d-%m-%Y")
if not os.path.exists(today):
    os.mkdir(today)
os.environ['FOLDER_NAME'] = today

##install

###Selenium web driver

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

###other

In [ ]:
!pip install requests-html
!pip install python-docx
# !pip install pip install excel-formulas-calculator
!git clone https://github.com/teploe-odealko/excel-formulas-calculator.git
!pip install html2text
!pip install python-pptx
!pip install -U kaleido


!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [ ]:
cd excel-formulas-calculator/

In [ ]:
!sudo python setup.py install

##import

In [ ]:
import openpyxl
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.formula import Tokenizer
from openpyxl.formula.translate import Translator
from openpyxl.utils.cell import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.formatting.rule import FormulaRule
from openpyxl.styles import Color, PatternFill, Font, Border

from typing import Dict, List, Tuple
from collections import defaultdict
from copy import copy
import glob
import re
import requests_html

import pandas as pd
import numpy as np

import html2text
import docx
from docx.shared import RGBColor
from bs4 import BeautifulSoup
import urllib3
from nltk.stem.snowball import SnowballStemmer
from datetime import datetime, date, timedelta
from efc.interfaces.iopenpyxl import OpenpyxlInterface
import plotly.express as px
import plotly.graph_objects as go
import pptx
from pptx import Presentation
from pptx.enum.text import PP_ALIGN

In [ ]:
cd ../

##constants

In [ ]:
WEEKLY_REPORT_DAY = 1
countries_map = {"United States": "США", "Diamond Princess": "Судно \"Diamond Princess\"",  "MS Zaandam":  "Судно \"MS Zaandam\"",
                "Taiwan*":"Тайвань", "Timor-Leste":"Восточный Тимор", "US":"США", "West Bank and Gaza":"Западный берег реки Иордан", "Syria": "Сирия", "Slovakia": "Словакия", "Sao Tome and Principe": "Сан-Томе и Принсипи", "Russia": "Россия",
                 "Korea, South": "Южная Корея", "Iran": "Иран", "Eswatini": "Эсватини", "Cote d'Ivoire": "Кот-д’Ивуар", "Congo (Kinshasa)": "ДР Конго", "Congo (Brazzaville)":"Республика Конго",
                 "Cabo Verde":"Кабо-Верде", "Burma": "Мьянма", "Afghanistan":  "Афганистан", "Albania":  "Албания", "Algeria":  "Алжир", "American Samoa":  "Американское Самоа", "Andorra":  "Андорра", "Angola":  "Ангола", "Anguilla": "Ангилья",
                 "Antarctica":  "Антарктика", "Antigua and Barbuda":  "Антигуа и Барбуда", "Argentina":  "Аргентина", "Armenia":  "Армения", "Aruba":  "Аруба", "Australia":  "Австралия", "Austria":  "Австрия", "Azerbaijan":  "Азербайджан", 
                 "Bahamas":  "Багамские острова", "Bahrain":  "Бахрейн", "Bangladesh":  "Бангладеш", "Barbados":  "Барбадос", "Belarus":  "Беларусь", "Belgium":  "Бельгия", "Belize":  "Белиз", "Benin":  "Бенин", 
                 "Bermuda": "Бермудские Острова", "Bhutan":  "Бутан", "Bolivia":  "Боливия", "Bosnia and Herzegovina":  "Босния и Герцеговина", "Botswana":  "Ботсвана", "Brazil":  "Бразилия", "Brunei":  "Бруней", "Bulgaria":  "Болгария",
                 "Burkina Faso":  "Буркина-Фасо", "Burundi":  "Бурунди", "Cambodia":  "Камбоджа", "Cameroon":  "Камерун", "Canada":  "Канада", "Cape Verde":  "Кабо-Верде", "Cayman Islands":  "Острова Кайман",
                 "Central African Republic":  "ЦАР", "Chad":  "Чад", "Chile":  "Чили", "China":  "Китай", "Colombia":  "Колумбия", "Comoros":  "Коморские острова",
                 "Cook Islands":  "Острова Кука", "Costa Rica":  "Коста-Рика",
                 "Ivory Coast (Cote d'Ivoire)":  "Кот-д’Ивуар", "Croatia":  "Хорватия", "Cuba":  "Куба", "Cyprus":  "Кипр", "Czechia":  "Чехия", "Denmark":  "Дания", "Djibouti":  "Джибути", "Dominica":  "Доминика",
                 "Dominican Republic":  "Доминиканская Республика", "East Timor (Timor-Leste)":  "Восточный Тимор", "Ecuador":  "Эквадор", "Egypt":  "Египет", "El Salvador":  "Сальвадор", "Equatorial Guinea":  "Экваториальная Гвинея",
                 "Eritrea":  "Эритрея", "Estonia":  "Эстония", "Ethiopia":  "Эфиопия", "Falkland Islands":  "Фолклендские острова", "Faroe Islands":  "Фарерские острова", "Fiji":  "Фиджи", "Finland":  "Финляндия", "France":  "Франция",
                 "French Guiana":  "Гвиана", "French Polynesia":  "Французская Полинезия", "Gabon":  "Габон", "Gambia":  "Гамбия", "Georgia":  "Грузия", "Germany":  "Германия", "Ghana":  "Гана", "Gibraltar":  "Гибралтар",
                 "Greece":  "Греция", "Greenland":  "Гренландия", "Grenada":  "Гренада", "Guadeloupe":  "Гваделупа", "Guam":  "Гуам", "Guatemala":  "Гватемала", "Guinea":  "Гвинея", "Guinea-Bissau":  "Гвинея-Бисау",
                 "Guyana":  "Гайана", "Haiti":  "Гаити", "Honduras":  "Гондурас", "Hong Kong":  "Гонконг", "Hungary":  "Венгрия", "Iceland":  "Исландия", "India":  "Индия", "Indonesia":  "Индонезия",
                 "Iran (Islamic Republic of)":  "Иран (Персия)", "Iraq":  "Ирак", "Ireland":  "Ирландия", "Israel":  "Израиль", "Italy":  "Италия", "Jamaica":  "Ямайка", "Japan":  "Япония", "Jordan":  "Иордания",
                 "Kazakhstan":  "Казахстан", "Kenya":  "Кения", "Kiribati":  "Кирибати", "Korea, Democratic People's Rep. (North Korea)":  "Северная Корея (КНДР)", "Korea, Republic of (South Korea)":  "Ю́жная Коре́я (Республика Корея)",
                 "Kosovo":  "Косово и Метохия", "Kuwait":  "Кувейт", "Kyrgyzstan":  "Киргизия", "Laos":  "Лаос", "Latvia":  "Латвия", "Lebanon":  "Ливан", "Lesotho":  "Лесото", "Liberia":  "Либерия",
                 "Libya":  "Ливия", "Liechtenstein":  "Лихтенштейн", "Lithuania":  "Литва", "Luxembourg":  "Люксембург", "Macau":  "Макао (Аомынь)", "North Macedonia":  "Северная Македония", "Madagascar":  "Мадагаскар", "Malawi":  "Малави",
                 "Malaysia":  "Малайзия", "Maldives":  "Мальдивы", "Mali":  "Мали", "Malta":  "Мальта", "Martinique":  "Мартиника", "Mauritania":  "Мавритания", "Mauritius":  "Маврикий", "Mayotte":  "Майотта", "Mexico":  "Мексика",
                 "Микронезия, Federated States of":  "Федеративные Штаты Микронезии", "Moldova":  "Молдавия", "Monaco":  "Монако", "Mongolia":  "Монголия", "Montenegro":  "Черногория", "Montserrat":  "Монтсеррат", 
                 "Morocco":  "Марокко", "Mozambique":  "Мозамбик", "Myanmar, Burma":  "Мьянма, (Би́рма)", "Namibia":  "Намибия", "Nauru":  "Науру", "Nepal":  "Непал", "Netherlands":  "Нидерланды", "New Caledonia":  "Новая Каледония", 
                 "New Zealand":  "Новая Зеландия", "Nicaragua":  "Никарагуа", "Niger":  "Нигер", "Nigeria":  "Нигерия", "Northern Mariana Islands":  "Северные Марианские", "":  "острова", "Norway":  "Норвегия", "Oman":  "Оман", 
                 "Pakistan":  "Пакистан", "Palau":  "Палау", "State of Palestine":  "Государство Палестина", "Panama":  "Панама", "Papua New Guinea":  "Папуа — Новая Гвинея", "Paraguay":  "Парагвай", "Peru":  "Перу", "Philippines":  "Филиппины", 
                 "Poland":  "Польша", "Portugal":  "Португалия", "Puerto Rico":  "Пуэрто-Рико", "Qatar":  "Катар", "Réunion":  "Реюньон", "Romania":  "Румыния", "Russian Federation":  "Россия, Росси́йская Федера́ция", "Rwanda":  "Руанда",
                 "Saint Kitts and Nevis":  "Сент-Китс и Невис", "Saint Lucia":  "Сент-Люсия", "Saint Vincent and the Grenadines":  "Сент-Винсент и Гренадины", "Samoa":  "Самоа", "San Marino":  "Сан-Марино", 
                 "Sao Tome and Príncipe":  "Сан-Томе и Принсипи", "Saudi Arabia":  "Саудовская Аравия", "Senegal":  "Сенегал", "Serbia":  "Сербия", "Seychelles":  "Сейшельские острова", "Sierra Leone":  "Сьерра-Леоне", 
                 "Singapore":  "Сингапур", "Slovakia (Slovak Republic)":  "Словакия", "Slovenia":  "Словения", "Solomon Islands":  "Соломоновы острова", "Somalia":  "Сомали", "South Africa":  "ЮАР", "South Sudan":  "Южный Судан", 
                 "Spain":  "Испания", "Sri Lanka":  "Шри-Ланка", "Sudan":  "Судан", "Suriname":  "Суринам", "Swaziland (Eswatini)":  "Сва́зиле́нд (Эсватини)", "Sweden":  "Швеция", "Switzerland":  "Швейцария", 
                 "Syria, Syrian Arab Republic":  "Сирия", "Taiwan":  "Китайская Республика (Тайвань)", "Tajikistan":  "Таджикистан", "Tanzania":  "Танзания", "Thailand":  "Таиланд", "Tibet":  "Тибет", 
                 "Timor-Leste (East Timor)":  "Восточный Тимор", "Togo":  "Того", "Tokelau":  "Токелау", "Tonga":  "Тонга", "Trinidad and Tobago":  "Тринидад и Тобаго", "Tunisia":  "Тунис", "Turkey":  "Турция", "Turkmenistan":  "Туркмения", 
                 "Tuvalu":  "Тувалу", "Uganda":  "Уганда", "Ukraine":  "Украина", "United Arab Emirates":  "ОАЭ", "United Kingdom":  "Великобритания", 
                 "Uruguay":  "Уругвай", "Uzbekistan":  "Узбекистан", "Vanuatu":  "Вануату", "Holy See":  "Папский Престол", "Venezuela":  "Венесуэла", "Vietnam":  "Вьетнам", 
                 "Virgin Islands (British)":  "Виргинские Острова (Великобритания)", "Virgin Islands (U.S.)":  "Виргинские Острова (США)", "Wallis and Futuna Islands":  "Уоллис и Футуна", "Western Sahara":  "Западная Сахара",
                 "Yemen":  "Йемен", "Zambia":  "Замбия", "Zimbabwe":  "Зимбабве"}

#Подготовка данных с репозитория университета Хопкинса

In [ ]:
!add-apt-repository ppa:git-core/ppa -y
!apt update; apt install git
!git clone --depth 1 --filter=blob:none --spars https://github.com/CSSEGISandData/COVID-19

In [ ]:
cd COVID-19/

In [ ]:
# !git sparse-checkout set csse_covid_19_data/csse_covid_19_daily_reports

In [ ]:
!git sparse-checkout set csse_covid_19_data/csse_covid_19_time_series

In [ ]:
cd ..

In [ ]:
confirmed_ts = pd.read_csv('/content/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_ts = pd.read_csv('/content/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_ts = pd.read_csv('/content/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [ ]:
confirmed_ts = confirmed_ts.groupby('Country/Region').sum().iloc[:,2:]
deaths_ts = deaths_ts.groupby('Country/Region').sum().iloc[:,2:]
recovered_ts = recovered_ts.groupby('Country/Region').sum().iloc[:,2:]

In [ ]:
li = []
for i in range(len(confirmed_ts)):
    tmp = pd.DataFrame()
    tmp['Дата'] = confirmed_ts.iloc[i,:].index
    tmp['Страна'] = confirmed_ts.iloc[i,:].name
    tmp['Заражений'] = confirmed_ts.iloc[i,:].values
    tmp['Заражений за день'] = tmp['Заражений'].diff(1)
    tmp['Заражений(МА)'] = tmp['Заражений за день'].rolling(7).sum()
    tmp['Смертей'] = deaths_ts.iloc[i,:].values
    tmp['Смертей за день'] = tmp['Смертей'].diff(1)
    tmp['Выздоровлений'] = recovered_ts.iloc[i,:].values
    tmp['Выздоровлений за день'] = tmp['Выздоровлений'].diff(1)
    li.append(tmp)

In [ ]:
df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
df = df[['Страна', 'Дата', 'Заражений', 'Заражений за день', 'Заражений(МА)',
       'Смертей', 'Смертей за день', 'Выздоровлений', 'Выздоровлений за день']]

In [ ]:
# path = r'/content/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports' # use your path
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename_csv in all_files:
#     df = pd.read_csv(filename_csv, index_col=None, header=0)
#     df['Last_Update'] = filename_csv.split('.')[0].split('/')[-1]
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)
# frame['Last_Update'] = pd.to_datetime(frame['Last_Update'], format='%m-%d-%Y')

In [ ]:
li = []
def translate_country(eng):
    try:
        return countries_map[eng]
    except KeyError:
        li.append(eng)
        return eng


df['Страна'] = df['Страна'].apply(translate_country)

In [ ]:
np.unique(np.array(li))

In [ ]:
np.unique(np.array(li))

In [ ]:
df['Дата'] = pd.to_datetime(df['Дата'], format='%m/%d/%y')

In [ ]:
df_diff = df

In [ ]:
df_diff

In [ ]:
# df = frame[['Confirmed', 'Recovered', 'Deaths', 'Country_Region', 'Last_Update', 'Combined_Key']]
# sorted_df = df.groupby(['Last_Update', 'Country_Region']).sum().reset_index().sort_values(by=['Country_Region', 'Last_Update'])


In [ ]:
# li = []
# for country in df['Country_Region'].unique():
#     country_df = df[df['Country_Region'] == country].copy()
#     country_df_diff = country_df[['Confirmed', 'Recovered', 'Deaths']].diff()
#     country_df['Confirmed_diff'] = country_df_diff['Confirmed']
#     country_df['Recovered_diff'] = country_df_diff['Recovered']
#     country_df['Deaths_diff'] = country_df_diff['Deaths']
#     country_df['Confirmed_rolling_sum'] = country_df_diff['Confirmed'].rolling(7).sum()
#     country_df['Confirmed_diff_rolling_sum'] = country_df['Confirmed_diff'].rolling(7).sum()
#     country_df['Deaths_diff_rolling_sum'] = country_df['Deaths_diff'].rolling(7).sum()
#     li.append(country_df)


# df_diff = pd.concat(li, axis=0, ignore_index=True)
# df_diff = df_diff.fillna(0)
# df_diff = df_diff[['Country_Region', 'Last_Update', 'Confirmed', 'Recovered', 'Deaths', 'Confirmed_diff', 'Recovered_diff', 'Deaths_diff', 'Confirmed_rolling_sum', 'Confirmed_diff_rolling_sum', 'Deaths_diff_rolling_sum']]
# df_diff.columns = ['Страна', 'Дата', 'Заражений', 'Выздоровлений', 'Смертей', 'Заражений за день', 'Выздоровлений за день', 'Смертей за день', 'Заражений(МА)', 'Заражений за день(МА)', 'Смертей за день(МА)']

#Срипт

##Utils

In [ ]:
def rename_input_filename(filename: str):
    today = datetime.today().strftime("%d-%m-%Y")
    splitted_filename = filename.split('.')
    splitted_filename[0] = f'{splitted_filename[0]}_{today}'
    output_filename = '.'.join(splitted_filename)
    return f'{today}/{output_filename}'

In [ ]:

# def continue_formula_n_down(ws: Worksheet, columns: list, row: int, n: int):
#     for column in columns:

#         formula = ws.cell(row=(row - n), column=column).value
#         last_cell_id = get_column_letter(column) + str(row - n)
#         next_cell_id = get_column_letter(column) + str(row)
#         ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)
#         ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


# def continue_formula_down(ws: Worksheet, columns: list, row: int):
#     continue_formula_n_down(ws, columns, row, 1)

In [ ]:
def continue_formula_n_down(ws: Worksheet, columns: list, row: int, n: int):
    for column in columns:

        formula = ws.cell(row=(row - n), column=column).value
        last_cell_id = get_column_letter(column) + str(row - n)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def continue_formula_down(ws: Worksheet, columns: list, row: int):
    continue_formula_n_down(ws, columns, row, 1)

In [ ]:
def  continue_date_down(ws: Worksheet, columns: list, days: int):
    # print('here')
    for column in columns:
        try:
            row = [c.value for c in ws[get_column_letter(column)]].index(None) + 1
        except ValueError:
            row = ws.max_row + 1
        last_table_date = ws.cell(row=row-1, column=column).value
        # print(row, column)
        # print(last_table_date)
        ws.cell(row=row, column=column).value = (last_table_date) + timedelta(days=days)
        ws.cell(row=row, column=column).number_format = 'DD.MM.YY'

        last_cell_id = get_column_letter(column) + str(row-1)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)

##МИР xlsx

In [ ]:
def base_countries_sheet(ws: Worksheet):
    for i, row in enumerate(dataframe_to_rows(df_diff, index=False)):
        ws.append(row)
        r = i + 2
        ws.cell(row=r, column=9).value = fr'=IF(YEAR(B{r})=2020,(IF(J{r}=1,WEEKNUM(B{r},2),IF(J{r}=2,WEEKNUM(B{r},2),IF(J{r}=3,WEEKNUM(B{r},2),IF(J{r}=4,WEEKNUM(B{r},2),IF(J{r}=5,WEEKNUM(B{r},2),IF(J{r}=6,WEEKNUM(B{r},2),IF(J{r}=7,WEEKNUM(B{r},2)+1)))))))),IF(YEAR(B{r})=2021,(IF(J{r}=1,WEEKNUM(B{r},2),IF(J{r}=2,WEEKNUM(B{r},2),IF(J{r}=3,WEEKNUM(B{r},2),IF(J{r}=4,WEEKNUM(B{r},2),IF(J{r}=5,WEEKNUM(B{r},2),IF(J{r}=6,WEEKNUM(B{r},2),IF(J{r}=7,WEEKNUM(B{r},2)+1))))))))+52))'
        ws.cell(row=r, column=10).value = fr'=WEEKDAY(B{r},2)'
        ws.cell(row=r, column=2).number_format = 'DD.MM.YYYY'
        
    ws.cell(row=1, column=9).value = fr'Номер недели'
    ws.cell(row=1, column=10).value = fr'Номер дня недели'

In [ ]:
def continue_number_down(ws: Worksheet, columns: list):
    for column in columns:

        row = ws.max_row + 1
        last_num = ws.cell(row=row - 1, column=column).value
        ws.cell(row=row, column=column).value = last_num + 1

        last_cell_id = get_column_letter(column) + str(row-1)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)

In [ ]:
def base_world_sheet(ws: Worksheet):
    new_row = ws.max_row + 1
    ws.cell(row=new_row, column=1).value = datetime.now()
    ws.cell(row=new_row, column=1).number_format = 'DD.MM.YY'
    continue_formula_down(ws, list(range(2, 6)) + [7, 8, 10, 11] + list(range(13, 21)), new_row)

    today = (datetime.today() - timedelta(days=1))
    daily_df = df_diff.groupby('Дата').sum().loc[today.strftime('%d.%m.%y')]
    ws.cell(row=new_row, column=6).value = daily_df['Заражений']
    ws.cell(row=new_row, column=9).value = daily_df['Выздоровлений']
    ws.cell(row=new_row, column=12).value = daily_df['Смертей']

In [ ]:
def weeks_sheet(ws: Worksheet):
    new_row = ws.max_row + 1
    # print(new_row)
    continue_number_down(ws, [1])
    continue_formula_down(ws, list(range(2, 15)), new_row)

In [ ]:
def countrie_sheet(ws: Worksheet):
    new_row = ws.max_row + 1
    # print(new_row)
    continue_number_down(ws, [1])
    continue_formula_down(ws, list(range(2, 32)), new_row)

In [ ]:
def world(filename: str):
    wb = openpyxl.load_workbook(filename)

    # clear sheet
    sheet_name = 'База страны'
    idx = wb.sheetnames.index(sheet_name)
    ws = wb[sheet_name]
    wb.remove(ws)
    wb.create_sheet(sheet_name, idx)


    base_countries_sheet(wb['База страны'])
    base_world_sheet(wb['База мир'])

    if (datetime.today().weekday() + 1 == WEEKLY_REPORT_DAY):
        weeks_sheet(wb['Недели'])
        for country in wb.sheetnames[8:-1]:
            countrie_sheet(wb[country])

    wb.save(rename_input_filename(filename))

##Россия xlsx

In [ ]:
URL = 'https://yastat.net/s3/milab/2020/covid19-stat/data/v10/deep_data.json'
REGION_COLUMN = 2
REGION_ROWS_RANGE = (2, 87)
WEEKLY_REPORT_DAY = 7
DAYS_OF_THE_WEEK_MAP = {
    1: 'ПН',
    2: 'ВТ',
    3: 'СР',
    4: 'ЧТ',
    5: 'ПТ',
    6: 'СБ',
    7: 'ВС',
}

In [ ]:
def get_general_info(info: dict):
    STOP_COVID_URL = 'https://стопкоронавирус.рф/information/'
    wd.get(STOP_COVID_URL)
    raw_rows = wd.find_elements_by_xpath('//h3[@class="cv-stats-virus__item-value"]')
    rows = [row.text for row in raw_rows]
    statistics = [int(''.join(row.strip('\n ').split())) for row in rows]
    useful_statistics = [statistics[2], statistics[4], statistics[0]]
    info.update({'general': useful_statistics})


def normolize_string(string: str):
    string = string.strip().lower().translate({ord('.'): None})
    string = ' '.join(string.split('-'))

    # remove stop words
    stop_words = ['г', 'обл', 'область', 'республика', 'автономная', 'ао']
    try:
        splitted_string = string.split()
    except IndexError:
        return ''
    stemmer = SnowballStemmer("russian")
    string = ' '.join(stemmer.stem(w) for w in splitted_string if w not in stop_words)
    return string


def fill_сases_sheet(ws: Worksheet, info: dict):
    OVERALL_CASES_FORMULA_ROW = 88

    last_available_date = datetime.strptime(info['dates'][-1], '%Y-%m-%d')
    last_column = ws.max_column
    last_table_date = ws.cell(row=1, column=last_column).value
    new_days_amount = (last_available_date - last_table_date).days
    for i in range(1, new_days_amount + 1):
        ws.cell(row=1, column=(last_column + i)).value = (last_table_date + timedelta(days=i))
        ws.cell(row=1, column=(last_column + i)).number_format = 'DD.MM.YYYY'

        for row in range(*REGION_ROWS_RANGE):
            name = ws.cell(row=row, column=REGION_COLUMN).value
            key = normolize_string(name)
            if key in info['data']:
                ws.cell(row=row, column=(last_column + i)).value = \
                    info['data'][key]['cases'][(i - 1) - new_days_amount][0]
            else:
                raise KeyError(key)

        formula = ws.cell(row=OVERALL_CASES_FORMULA_ROW, column=ws.max_column - 1).value
        last_cell_id = get_column_letter(ws.max_column - 1) + str(OVERALL_CASES_FORMULA_ROW)
        next_cell_id = get_column_letter(ws.max_column) + str(OVERALL_CASES_FORMULA_ROW)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)

    info.update({'new_days_amount': new_days_amount})


def get_regions_info() -> dict:
    session = requests_html.HTMLSession()
    final_region_data = {}
    response: dict = session.get(URL, verify=False).json()
    regions = response['russia_stat_struct']['data']
    dates = response['russia_stat_struct']['dates']
    for region in regions:
        full_name = regions[region]['info']['name']
        short_name = regions[region]['info']['short_name']
        regions[region]['info'].pop('name', None)
        regions[region]['info'].pop('short_name', None)
        final_region_data.update(
            dict.fromkeys([normolize_string(full_name), normolize_string(short_name)], regions[region]))
    return {'data': final_region_data,
            'dates': dates,
            'last_date': datetime.strptime(dates[-1], '%Y-%m-%d')}


def continue_formula_right(ws: Worksheet, rows: list, column: int):
    for row in rows:
        formula = ws.cell(row=row, column=column - 1).value
        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def  continue_date_right(ws: Worksheet, rows: list):
    for row in rows:
        try:
            column = [c.value for c in ws[row]][3:].index(None) + 4
        except ValueError:
            column = ws.max_column + 1
        last_table_date = ws.cell(row=row, column=column - 1).value
        ws.cell(row=row, column=column).value = (last_table_date) + timedelta(days=1)
        ws.cell(row=row, column=column).number_format = 'DD.MM.YYYY'

        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def continue_number_right(ws: Worksheet, rows: list):
    for row in rows:
        try:
            column = [c.value for c in ws[row]].index(None) + 1
        except ValueError:
            column = ws.max_column + 1
        last_num = ws.cell(row=row, column=column - 1).value
        ws.cell(row=row, column=column).value = last_num + 1

        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def gain_sheet(ws: Worksheet, info: dict):
    NEW_COLUMN_NUM_GAIN = [c.value for c in ws[1]].index(None) + 1

    for i in range(info['new_days_amount']):
        ws.insert_cols(NEW_COLUMN_NUM_GAIN+i)
        continue_date_right(ws, [1])
        continue_formula_right(ws, list(range(2, 96)), NEW_COLUMN_NUM_GAIN + i)

    info['last_col_in_gain_sheet'] = [c.value for c in ws[1]].index(None)


def daily_region_gain_sheet(ws: Worksheet, info: dict):
    NEW_COLUMN_NUM_DAILY_GAIN = ws.max_column + 1

    for i in range(info['new_days_amount']):
        continue_date_right(ws, [1, 20, 35])
        continue_formula_right(ws,
                               list(range(2, 19)) + list(range(21, 34)) + list(range(36, 49)),
                               NEW_COLUMN_NUM_DAILY_GAIN + i)


def weekly_gain_sheet(ws: Worksheet, info: dict):
    new_col = [c.value for c in ws[1]].index(None) + 1
    ws.insert_cols(new_col)
    continue_number_right(ws, [1])
    continue_formula_right(ws, list(range(2, 94)), new_col)
    for row in range(2, 87):
        # left part
        raw_formula = ws.cell(row=row, column=new_col).value
        tokenized_formula = Tokenizer(raw_formula)
        last_col_in_gain_sheet = info['last_col_in_gain_sheet']

        tokenized_formula.items[1].value = re.sub(r'(?<=:\$)\D*',
                                                  get_column_letter(last_col_in_gain_sheet),
                                                  tokenized_formula.items[1].value)
        tokenized_formula.items[3].value = re.sub(r'(?<=:\$).*(?=\$)',
                                                  get_column_letter(last_col_in_gain_sheet),
                                                  tokenized_formula.items[3].value)
        new_formula = ''.join(token.value for token in tokenized_formula.items)
        ws.cell(row=row, column=new_col).value = fr'={new_formula}'

        # right part
        ws.cell(row=row, column=new_col+2).value = fr'=MAX($C{row}:${get_column_letter(new_col)}{row})'
        ws.cell(row=row, column=new_col+3).value = fr'={get_column_letter(new_col+2)}{row}-{get_column_letter(new_col)}{row}'
        ws.cell(row=row, column=new_col+4).value = fr'={get_column_letter(new_col)}{row}-{get_column_letter(new_col-1)}{row}'
        ws.cell(row=row, column=new_col+6).value = fr'={get_column_letter(new_col)}{row}/{get_column_letter(new_col-1)}{row}-100%'
        ws.cell(row=row, column=new_col+7).value = fr'={get_column_letter(new_col-1)}{row}/{get_column_letter(new_col-2)}{row}-100%'


    ws.cell(row=87, column=new_col+4).value = fr'={get_column_letter(new_col)}{87}-{get_column_letter(new_col-1)}{87}'


def tpr_weekly_gain_sheet(ws: Worksheet, info: dict):
    new_col = [c.value for c in ws[1]].index(None) + 1
    ws.insert_cols(new_col)
    continue_number_right(ws, [1])
    continue_formula_right(ws, list(range(2, 88)), new_col)



def get_first_clear_row_in_column(column):
    return max((c.row for c in column if c.value is not None)) + 1

def base_rf_sheet(ws: Worksheet, info: dict):
    NEW_ROW_NUM = get_first_clear_row_in_column(ws['B'])

    ws.cell(row=NEW_ROW_NUM, column=1).value = info['last_date']
    ws.cell(row=NEW_ROW_NUM, column=1).number_format = 'DD.MM'
    for i, stat in enumerate(info['general']):
        ws.cell(row=NEW_ROW_NUM, column=2 + i).value = info['general'][i]
        ws.cell(row=NEW_ROW_NUM, column=2 + i)._style = copy(ws.cell(row=NEW_ROW_NUM - 1, column=2 + i)._style)
    continue_formula_down(ws,
                          list(range(5, 20)) + [26] + list(range(28, 36)),
                          NEW_ROW_NUM)

    if info['last_date'].weekday() + 1 == WEEKLY_REPORT_DAY:  # if it's sunday
        continue_formula_n_down(ws, list(range(20, 25))+[27], NEW_ROW_NUM, 7)
        continue_formula_n_down(ws, list(range(20, 23)), NEW_ROW_NUM + 1, 7)
    # I_column = [c.value for c in ws['I'][10:]]
    # try:
    #     info['day_infections_cell'] = f'I{I_column.index(None)+10}'
    # except ValueError:
    #     info['day_infections_cell'] = f'I{len(I_column)+10}'


def date_week_sheet(ws: Worksheet, info: dict):
    NEW_ROW_NUM = get_first_clear_row_in_column(ws['A'])
    ws.cell(row=NEW_ROW_NUM, column=1).value = ws.cell(row=NEW_ROW_NUM - 1, column=1).value + 1
    continue_date_down(ws, list(range(2, 4)), 7)




def parse_info():
    info = get_regions_info()
    get_general_info(info)
    return info


def weekly_region_gain_sheet(ws: Worksheet, info: dict):
    new_column = ws.max_column + 1

    continue_number_right(ws, [1, 20])
    continue_formula_right(ws,
                           list(range(2, 17)) + list(range(21, 34)),
                           new_column)

def gain_7day_sheet(ws: Worksheet, info: dict):
    new_column = [c.value for c in ws[1]].index(None) + 1

    for i in range(info['new_days_amount']):
        ws.insert_cols(new_column+i)
        continue_date_right(ws, [1])
        continue_formula_right(ws, list(range(2, 94)), new_column + i)

    for row in range(2, 87):
        ws.cell(row=row, column=new_column+i+3).value = fr'={get_column_letter(new_column+i)}{row}'
        ws.cell(row=row, column=new_column+i+2).value = fr'=RANK({get_column_letter(new_column+i+3)}{row},${get_column_letter(new_column+i+3)}$2:${get_column_letter(new_column+i+3)}$86,0)'
        ws.cell(row=row, column=new_column+i+4).value = fr'={get_column_letter(new_column+i)}{row}-{get_column_letter(new_column+i-7)}{row}'
        ws.cell(row=row, column=new_column+i+6).value = fr'={get_column_letter(new_column+i)}{row}/{get_column_letter(new_column+i-7)}{row}-1'
    ws.cell(row=row, column=new_column+i+3).value = fr'={get_column_letter(new_column + i)}{row}'

    for row in range(92, 97):
        ws.cell(row=row, column=new_column+i+4).value = fr'=VLOOKUP({get_column_letter(new_column+i+2)}{row},${get_column_letter(new_column+i+2)}$2:${get_column_letter(new_column+i+3)}$86,2,0)'
        ws.cell(row=row, column=new_column+i+5).value = fr'={get_column_letter(new_column+i+4)}{row}/${get_column_letter(new_column+i+4)}$97'
        ws.cell(row=row, column=new_column+i+6).value = fr'==VLOOKUP({get_column_letter(new_column+i+2)}{row},${get_column_letter(new_column+i+2)}$2:${get_column_letter(new_column+i+6)}$86,4,0)'
    ws.cell(row=97, column=new_column+i+4).value = fr'={get_column_letter(new_column+i+3)}87'
    ws.cell(row=97, column=new_column+i+5).value = fr'={get_column_letter(new_column+i+4)}{97}/${get_column_letter(new_column+i+4)}$97'

def rt_sheet(ws: Worksheet, info: dict):
    new_column = [c.value for c in ws[2]][3:].index(None) + 4
    for i in range(info['new_days_amount']):
        ws.insert_cols(new_column+i)
        continue_date_right(ws, [2])
        continue_formula_right(ws, list(range(3, 89)), new_column + i)

    ws.cell(row=1, column=(new_column+i+5)).value = info['last_date']
    c = get_column_letter(new_column+i+5)
    r = '1'
    cell1 = c+r
    ws.cell(row=1, column=(new_column+i+6)).value = fr'=MATCH({cell1},$A$2:{get_column_letter(new_column+i)}2)-3'

    for row in range(3, 88):
        col = get_column_letter(new_column+i)
        ws.cell(row=row, column=(new_column+i+2)).value = fr'=RANK({col}{row},${col}$3:${col}$87,0)'

        ws.cell(row=row, column=(new_column+i+4)).value = fr'=IF(INDEX($D$3:${col}$87,${get_column_letter(new_column+i+8)}{row},${get_column_letter(new_column+i+6)}$1)<1,1,0)'
        ws.cell(row=row, column=(new_column+i+5)).value = fr'=IF(INDEX($D$3:${col}$87,${get_column_letter(new_column+i+8)}{row},${get_column_letter(new_column+i+6)}$1)>1,1,0)'
        ws.cell(row=row, column=(new_column+i+6)).value = fr'=IF(INDEX($D$3:${col}$87,${get_column_letter(new_column+i+8)}{row},${get_column_letter(new_column+i+6)}$1)=1,1,0)'

    ws.cell(row=88, column=(new_column+i+4)).value = fr'=SUM({get_column_letter(new_column+i+4)}3:{get_column_letter(new_column+i+4)}87)'
    ws.cell(row=88, column=(new_column+i+5)).value = fr'=SUM({get_column_letter(new_column+i+5)}3:{get_column_letter(new_column+i+5)}87)'
    ws.cell(row=88, column=(new_column+i+6)).value = fr'=SUM({get_column_letter(new_column+i+6)}3:{get_column_letter(new_column+i+6)}87)'
    ws.cell(row=89, column=(new_column+i+6)).value = fr'=85-SUM({get_column_letter(new_column+i+4)}88:{get_column_letter(new_column+i+6)}88)'
    info['rt_bigger1_cell'] = f'{get_column_letter(new_column+i+5)}88'


def delta_day_sheet(ws: Worksheet, info: dict):
    new_column = ws.max_column + 1
    bigger_zero_fill = PatternFill(start_color='F1CCB1', end_color='F1CCB1', fill_type='solid')
    lower_zero_fill = PatternFill(start_color='E4EFDC', end_color='E4EFDC', fill_type='solid')

    for i in range(info['new_days_amount']):
        continue_date_right(ws, [1])
        ws.cell(row=2, column=new_column + i).value = DAYS_OF_THE_WEEK_MAP.get(
            ws.cell(row=1, column=new_column + i).value.weekday() + 1
        )
        last_cell_id = get_column_letter(new_column + i - 1) + str(2)
        next_cell_id = get_column_letter(new_column + i) + str(2)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)

        continue_formula_right(ws, list(range(3, 98)), new_column + i)
        cell_ids_formatting = '{}3:{}87'.format(get_column_letter(new_column + i), get_column_letter(new_column + i))
        ws.conditional_formatting.add(cell_ids_formatting, FormulaRule(formula=[f'{next_cell_id}>0'], fill=bigger_zero_fill))
        ws.conditional_formatting.add(cell_ids_formatting, FormulaRule(formula=[f'{next_cell_id}<0'], fill=lower_zero_fill))


def msc_sp_sheet(ws: Worksheet, info: dict):
    new_column = ws.max_column + 1
    # bigger_zero_fill = PatternFill(start_color='F1CCB1', end_color='F1CCB1', fill_type='solid')
    # lower_zero_fill = PatternFill(start_color='E4EFDC', end_color='E4EFDC', fill_type='solid')

    for i in range(info['new_days_amount']):
        continue_date_right(ws, [1])
        ws.cell(row=2, column=new_column + i).value = DAYS_OF_THE_WEEK_MAP.get(
            ws.cell(row=1, column=new_column + i).value.weekday() + 1
        )
        last_cell_id = get_column_letter(new_column + i - 1) + str(2)
        next_cell_id = get_column_letter(new_column + i) + str(2)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)
        continue_formula_right(ws, list(range(3, 98)), new_column + i)


In [ ]:
def russia(filename: str):
    wb = openpyxl.load_workbook(filename)

    info = parse_info()

    fill_сases_sheet(wb['Случаев'], info)
    gain_sheet(wb['Прирост'], info)
    msc_sp_sheet(wb['МСК и СП'], info)
    rt_sheet(wb['Rt'], info)
    gain_7day_sheet(wb['прирост 7дн'], info)
    delta_day_sheet(wb['дельта за сутки'], info)
    daily_region_gain_sheet(wb['По рег прис (сут)'], info)

    if info['last_date'].weekday()+1 == WEEKLY_REPORT_DAY:
        weekly_gain_sheet(wb['Прирост нед'], info)
        tpr_weekly_gain_sheet(wb['Тпр нед прироста'], info)
        weekly_region_gain_sheet(wb['По рег прис (нед)'], info)
        date_week_sheet(wb['Дата-неделя'], info)

    base_rf_sheet(wb['База РФ'], info)

    wb.save(rename_input_filename(filename))

##Сводка docx

In [ ]:
# world_out_file = 'МИР21.xlsx'
# russia = '/content/russia_regions (1).xlsx'

In [ ]:
def choise(var: int, s1, s2, s3):
    if var > 0:
        return s1
    elif var < 0:
        return s2
    else:
        return s3

In [ ]:
def paragraph1(ws: Worksheet, doc, interface):
    row = ws.max_row
    print(row)
    week_to_week = int(interface.calc_cell(f'P{row}', 'База мир')*100)

    w1 = choise(week_to_week,
                    'ускоряется',
                    'замедляется',
                    'остается прежним')

    par = f'Темп роста количества новых случаев заражения {w1} (прирост за последние 7 дней находится на уровне {week_to_week}%).'
    doc.add_paragraph(par, style='List Bullet')

In [ ]:
def paragraph2(ws: Worksheet, doc, interface):
    # row = [c.value for c in ws['AC'][7:]].index(None) + 7
    AC_column = [c.value for c in ws['AC']]
    while AC_column[-1] is None:
        AC_column = AC_column[:-1]
    row = len(AC_column)
    week = interface.calc_cell(f'AC{row}', 'База РФ')
    week_to_week = int(interface.calc_cell(f'AD{row}', 'База РФ')*100)
    week_to_week_prev = int(interface.calc_cell(f'AD{row-1}', 'База РФ')*100)

    w1 = choise(week_to_week,
                    'наблюдается динамика увеличения заболеваемости',
                    'наблюдается динамика уменьшения заболеваемости',
                    'динамика заболеваемости остается на прежнем уровне')

    if week_to_week < week_to_week_prev:
        w2 = 'замедлились'
    elif week_to_week > week_to_week_prev:
        w2 = 'ускорились'
    else:
        w2 = 'остались неизменными'

    dynamic = interface.calc_cell(f'E{row}', 'База РФ')
    dynamic_speed = int(interface.calc_cell(f'R{row}', 'База РФ')*100)
    dynamic_speed_prev = int(interface.calc_cell(f'R{row-1}', 'База РФ')*100)

    w3 = choise(dynamic_speed,
                f'повысился на {abs(dynamic_speed)}%',
                f'снизился на {abs(dynamic_speed)}%',
                'остался неизменным')
    w4 = choise(dynamic_speed_prev,
                f'повысился на {abs(dynamic_speed_prev)}%',
                f'снизился на {abs(dynamic_speed_prev)}%',
                'остался неизменным')

    recover = interface.calc_cell(f'AE{row}', 'База РФ')
    recover_week_to_week = int(interface.calc_cell(f'AF{row}', 'База РФ')*100)

    mortality = interface.calc_cell(f'AG{row}', 'База РФ')
    mortality_week_to_week = int(interface.calc_cell(f'AH{row}', 'База РФ')*100)

    w5 = choise(recover_week_to_week,
                f'вырасло на {abs(dynamic_speed)}%',
                f'снизилось на {abs(dynamic_speed)}%',
                'осталось неизменным')
    
    w6 = choise(mortality_week_to_week,
                f'вырос на {abs(dynamic_speed)}%',
                f'снизился на {abs(dynamic_speed)}%',
                'остался неизменным')

    day_deaths = interface.calc_cell(f'H{row}', 'База РФ')
    max_day_deaths = interface.calc_cell(f'L{row}', 'База РФ')
    is_daily_record = ', что является новым суточным максимумом за весь период пандемии.' if day_deaths == max_day_deaths else '.' 

    par = f'В России {w1}, темпы недельного роста {w2}: количество заболевших за последние 7 дней увеличилось на {week_to_week}% (аналогичный показатель днем ранее - {week_to_week_prev}%) и составило {week} случаев. '\
        f'Темп суточной динамики {w3} (днем ранее {w4}), за истекшие сутки зарегистрировано {dynamic} случаев(ая). '\
        f'Количество выздоровлений за семидневный период {w5} по сравнению с показателем предшествующей недели и составило {recover} случаев(ая). '\
        f'Недельный прирост количества летальных исходов {w6} и составил {mortality} случаев(ая). '\
        f'Суточный показатель составил {day_deaths} случаев(ая){is_daily_record}'
    doc.add_paragraph(par, style='List Bullet')

In [ ]:
def paragraph3(ws: Worksheet, doc, interface):
    col = ws.max_column 
    gain = interface.calc_cell(f'{get_column_letter(col)}3', 'МСК и СП')
    gain_speed = int(interface.calc_cell(f'{get_column_letter(col)}6', 'МСК и СП')*100)
    gain_speed_prev = int(interface.calc_cell(f'{get_column_letter(col-1)}6', 'МСК и СП')*100)

    gain_week = int(interface.calc_cell(f'{get_column_letter(col)}13', 'МСК и СП')*100)
    gain_week_prev = int(interface.calc_cell(f'{get_column_letter(col-1)}13', 'МСК и СП')*100)

    w1 = choise(gain_speed,
                'зафиксировано повышение суточной динамики',
                'зафиксировано снижение суточной динамики',
                'суточный прирост не изменился')
    
    w2 = choise(gain_speed,
                f'повысился на {abs(gain_speed)}%',
                f'снизился на {abs(gain_speed)}%',
                f'остался неизменным')
    
    w3 = choise(gain_speed_prev,
                f'повышение составило {gain_speed_prev}%',
                f'снижение составило {gain_speed_prev}%',
                f'темп остался неизменным')
    
    w4 = choise(gain_week,
                f'повышения недельного прироста составляет {abs(gain_week)}%',
                f'снижения недельного прироста составляет {abs(gain_week)}%',
                f'недельного прироста остался неизменным')
    
    w5 = choise(gain_week_prev,
                f'повышениe {abs(gain_week_prev)}%',
                f'снижениe {abs(gain_week_prev)}%',
                f'темп был неизменным')

    par = f'В Москве {w1}: за истекший день темп прироста {w2} (днем ранее {w3}), число выявленных случаев составило {gain}. '\
        f'Темп {w4} (днем ранее {w5}).'

    doc.add_paragraph(par, style='List Bullet')



In [ ]:
def paragraph4(ws: Worksheet, doc, interface):
    col = ws.max_column 
    gain = interface.calc_cell(f'{get_column_letter(col)}4', 'МСК и СП')
    gain_speed = int(interface.calc_cell(f'{get_column_letter(col)}7', 'МСК и СП')*100)
    gain_speed_prev = int(interface.calc_cell(f'{get_column_letter(col-1)}7', 'МСК и СП')*100)

    gain_week = int(interface.calc_cell(f'{get_column_letter(col)}15', 'МСК и СП')*100)
    gain_week_prev = int(interface.calc_cell(f'{get_column_letter(col-1)}15', 'МСК и СП')*100)

    w1 = choise(gain_speed,
                'зафиксировано повышение суточной динамики',
                'зафиксировано снижение суточной динамики',
                'суточный прирост не изменился')
    
    w2 = choise(gain_speed,
                f'повысился на {abs(gain_speed)}%',
                f'снизился на {abs(gain_speed)}%',
                f'остался неизменным')
    
    w3 = choise(gain_speed_prev,
                f'повышение составило {gain_speed_prev}%',
                f'снижение составило {gain_speed_prev}%',
                f'темп остался неизменным')
    
    w4 = choise(gain_week,
                f'повышения недельного прироста составляет {abs(gain_week)}%',
                f'снижения недельного прироста составляет {abs(gain_week)}%',
                f'недельного прироста остался неизменным')
    
    w5 = choise(gain_week_prev,
                f'повышениe {abs(gain_week_prev)}%',
                f'снижениe {abs(gain_week_prev)}%',
                f'темп был неизменным')

    par = f'В Санкт-Петербурге {w1}: за истекший день темп прироста {w2} (днем ранее {w3}), число выявленных случаев составило {gain}. '\
        f'Темп {w4} (днем ранее {w5}).'

    doc.add_paragraph(par, style='List Bullet')


In [ ]:
def paragraph5(ws: Worksheet, doc, interface):
    info = defaultdict(list)
    col = [c.value for c in ws[1]].index(None) + 6
    # cities = []
    # percents = []
    # print(get_column_letter(col))
    for i in range(2, 87):
        info['ratio'].append(interface.calc_cell(f'{get_column_letter(col)}{i}', 'прирост 7дн'))
        info['regions'].append(interface.calc_cell(f'{get_column_letter(col-1)}{i}', 'прирост 7дн'))
        info['cases'].append(interface.calc_cell(f'{get_column_letter(col-3)}{i}', 'прирост 7дн'))
    # print(info['cases'])
    top5_idx = np.array(info['cases']).argsort()[-5:][::-1]
    top5_regions = np.array(info['regions'])[top5_idx]
    top5_cases = np.array(info['cases'])[top5_idx]

    total = interface.calc_cell(f'{get_column_letter(col-3)}{87}', 'прирост 7дн')
    top5_percents = [case/total for case in top5_cases]

    top5_list = [f'{region} ({round(ratio*100)}%)' for region, ratio in zip(top5_regions, top5_percents)]
    # print(top5_list)
    par = f'Регионы, которые обеспечили основной прирост по России за последние 7 дней:\n{", ".join(top5_list)}.'


    doc.add_paragraph(par, style='List Bullet')

In [ ]:
def paragraph6(ws: Worksheet, doc, interface):
    info = defaultdict(list)
    col = [c.value for c in ws[1]].index(None) + 6
    # cities = []
    # percents = []
    for i in range(2, 87):
        info['ratio'].append(interface.calc_cell(f'{get_column_letter(col)}{i}', 'прирост 7дн'))
        info['regions'].append(interface.calc_cell(f'{get_column_letter(col-1)}{i}', 'прирост 7дн'))
        info['last_date'].append(interface.calc_cell(f'{get_column_letter(col-3)}{i}', 'прирост 7дн'))
    top20 = np.array(info['last_date']).argsort()[-20:][::-1]
    top20_regions = np.array(info['regions'])[top20]
    top20_ratio = np.array(info['ratio'])[top20]
    top5_ratio_index = top20_ratio.argsort()[-5:][::-1]
    top5_regions = top20_regions[top5_ratio_index]
    top5_ratio = top20_ratio[top5_ratio_index]
    top5_list = [f'{region} (рост на {round(ratio*100)}%)' for region, ratio in zip(top5_regions, top5_ratio)]
    # print(top5_list)

    bigger50_idx = [i for i, value in enumerate(info['ratio']) if round(value*100) >= 50]
    bigger50_regions = np.array(info['regions'])[bigger50_idx]

    

    if len(bigger50_regions) > 0:
        par2 = f'\nСтоит также отметить, что дополнительно к 5 названным еще в {len(bigger50_regions)} регионе(ах) с меньшей статистикой* скорость недельного прироста превышает 50%.\n'\
                f'* {", ".join(bigger50_regions)}'
    else:
        par2 = ''
    
    par1 = f'Скорость прироста от недели к неделе (из числа ТОП-20 по абсолютному приросту за последние 7 дней): {", ".join(top5_list)}.'
    doc.add_paragraph(par1+par2, style='List Bullet')

In [ ]:
def fill_header(doc):
    header = '[В Мире и России]'
    description = 'Файл «МИР».\n'\
                    'Лист «Источник» - ссылка на исходные данные. Скачиваем данные в формате xls. Загружаем в лист «База страны».\n'\
                    'Лист «База мир». В столбце А проставить текущую дату, протянуть формулы по всей строке.\n'\
                    'Лист «Сутки_забол». Задаем формулы для формирования данных за сутки и за последние 7 дней. Дата для формирования данных за сутки задается как Т-1 (дата предыдущего дня).\n'\
                    'Данные по вакцинации берем с сайта GOGOV.ru вакцинация. Других источников таких данных пока не обнаружили.\n'\
                    'Иногда случаются «выбросы» по отдельным странам, когда пересматривается подсчет показателей и за один день показывается статистика за целый период. В этом случае ищем «выброс» в листе «База страны», по какой стране произошло нестандартное отклонение.'
    p1 = doc.add_paragraph()
    p1.alignment = 1
    font = p1.add_run(header).font
    font.color.rgb = RGBColor.from_string('EB4034')
    
    p2 = doc.add_paragraph()
    font = p2.add_run(description).font
    font.color.rgb = RGBColor.from_string('A32E26')



In [ ]:
RBC_URL = 'https://www.rbc.ru/story/5e2881539a794724ab627caa'
wd.get(RBC_URL)
news_links = wd.find_elements_by_xpath("//a[@class='item__link']")
headers = wd.find_elements_by_xpath("//span[@class='item__title rm-cm-item-text']")

for i, header in enumerate(headers):
    try:
        if 'о пандемии на' in (header.text.split('.')[1]):
            break
    except IndexError:
        continue

news_links[i].click()
elem = wd.find_element_by_xpath("//*")
source_code = elem.get_attribute("outerHTML")

h = html2text.HTML2Text()
h.ignore_links = True
source_text = h.handle(source_code)
rbc_news = [s[4:] for s in source_text.split('\n') if s.startswith('  * ')]

In [ ]:
STOPCORONA_URL = 'https://стопкоронавирус.рф/news/'
wd.get(STOPCORONA_URL)

stopcorona_headers_elements = wd.find_elements_by_xpath("//div[@class='cv-news-page__news-list-item']//h2")
stopcorona_headers = [header.text for header in stopcorona_headers_elements]


In [ ]:
STOPCORONA_URL_BUISNESS = 'https://стопкоронавирус.рф/news/?tags=1'
wd.get(STOPCORONA_URL_BUISNESS)

stopcorona_headers_elements = wd.find_elements_by_xpath("//div[@class='cv-news-page__news-list-item']//h2")
stopcorona_headers_buisness = [header.text for header in stopcorona_headers_elements]


In [ ]:
YANDEX_URL = 'https://yandex.ru/news/rubric/koronavirus'
wd.get(YANDEX_URL)

stopcorona_headers_elements = wd.find_elements_by_xpath("//div[@class='mg-card__annotation']")
yandex_annotation = [header.text for header in stopcorona_headers_elements if header.text != '']


In [ ]:
def paragraph_news(doc):
    doc.add_paragraph('Справочно')
    p = doc.add_paragraph()
    runner = p.add_run('РБК')
    runner.bold = True
    for news in rbc_news:
        doc.add_paragraph(news, style='List Bullet')

    p = doc.add_paragraph()
    runner = p.add_run('Стопкоронавирус')
    runner.bold = True
    for news in stopcorona_headers[:10]:
        doc.add_paragraph(news, style='List Bullet')

    p = doc.add_paragraph()
    runner = p.add_run('Стопкоронавирус по тегу "бизнесу"')
    runner.bold = True
    for news in stopcorona_headers_buisness[:10]:
        doc.add_paragraph(news, style='List Bullet')

    p = doc.add_paragraph()
    runner = p.add_run('Яндекс')
    runner.bold = True
    for news in yandex_annotation[:10]:
        doc.add_paragraph(news, style='List Bullet')

In [ ]:
def report_docx():
    wb_world = openpyxl.load_workbook(rename_input_filename(world_input_filename))
    wb_russia = openpyxl.load_workbook(rename_input_filename(russia_input_filename))
    interface_world = OpenpyxlInterface(wb=wb_world, use_cache=True)
    interface_russia = OpenpyxlInterface(wb=wb_russia, use_cache=True)

    doc = docx.Document()
    fill_header(doc)
    paragraph1(wb_world['База мир'], doc, interface_world)
    paragraph2(wb_russia['База РФ'], doc, interface_russia)
    paragraph3(wb_russia['МСК и СП'], doc, interface_russia)
    paragraph4(wb_russia['МСК и СП'], doc, interface_russia)
    paragraph5(wb_russia['прирост 7дн'], doc, interface_russia)
    paragraph6(wb_russia['прирост 7дн'], doc, interface_russia)
    paragraph_news(doc)
    doc.save(rename_input_filename('сводка.docx'))


#Графики

In [ ]:
!wget https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv

In [ ]:
vac = pd.read_csv('/content/vaccinations.csv')
vac['location'] = vac['location'].apply(translate_country)

In [ ]:
last_date = df_diff['Дата'].sort_values().iloc[-1]
top5_countries = df_diff[df_diff['Дата'] == last_date].sort_values('Заражений(МА)', ascending=False).iloc[:5]['Страна'].values

In [ ]:
if not os.path.exists(f'{today}/graphics'):
    os.mkdir(f'{today}/graphics')

In [ ]:
prs.slides[0].shapes[7].table.cell(0, 1).text

In [ ]:
pic_width = 7.34
pic_height = 2.98

#(slide, x, y)
pic_positions = [(1, 1.33, 2.65), (1, 1.33, 5.84), (1, 1.33, 8.86),
                 (1, 8.89, 2.65), (1, 8.89, 5.84), (1, 8.89, 8.86),
                 (1, 16.79, 2.65), (1, 16.79, 5.84), (1, 16.79, 8.86),
                 (2, 5.02, 2.59), (2, 5.02, 5.88), (2, 5.02, 8.86),
                 (2, 13.74, 2.59), (2, 13.74, 5.88), (2, 13.74, 8.86)]
county_name_positions = [(1.5, 2.14), (8.9, 2.14), (16.8, 2.14), (5.05, 2.14), (13.66, 2.14)]
confirmed_text_positions = [(1.61, 2.6), (9, 2.6), (16.87, 2.6), (5.1, 2.6), (13.7, 2.6)]
deaths_text_positions = [(1.61, 5.84), (9, 5.84), (16.87, 5.84), (5.1, 5.95), (13.7, 5.95)]
vaccine_text_positions = [(1.61, 9.04), (9, 9.04), (16.87, 9.04), (5.1, 9.04), (13.7, 9.04)]
prs = Presentation(prs_filename)


for i, country in enumerate(top5_countries):
    # First graphic (confirmed)
    country_df = df_diff[df_diff['Страна'] == country]
    fig = px.line(country_df.sort_values('Дата'),
                  x="Дата",
                  y="Заражений за день",
                  color_discrete_sequence=["red"],
                  template='plotly_white',)
    fig.update_layout(
        margin=dict(t=0,l=0,b=0,r=15)
    )
    fig.update_yaxes(title_text="")
    fig.update_xaxes(title_text="")
    pic_path = f'{today}/graphics/Заражений {country}.jpeg'
    fig.write_image(pic_path)
    
    slide, x, y = pic_positions.pop(0)
    prs.slides[pic_pos[0]].shapes.add_picture(pic_path, pptx.util.Cm(x), pptx.util.Cm(y),
                               width=pptx.util.Cm(pic_width), height=pptx.util.Cm(pic_height))

    # country name
    x, y = county_name_positions.pop(0)
    txBox = prs.slides[pic_pos[0]].shapes.add_textbox(pptx.util.Cm(x), pptx.util.Cm(y),
                               width=pptx.util.Cm(1), height=pptx.util.Cm(1))
    tf = txBox.text_frame
    p = tf.paragraphs[0]
    p.text = country
    p.font.size = pptx.util.Pt(10)
    p.font.bold = True

    # Confirmed cases text
    x, y = confirmed_text_positions.pop(0)
    txBox = prs.slides[slide].shapes.add_textbox(pptx.util.Cm(x), pptx.util.Cm(y),
                               width=pptx.util.Cm(1), height=pptx.util.Cm(1))
    tf = txBox.text_frame
    p = tf.paragraphs[0]
    confirmed = country_df[country_df['Дата'] == country_df['Дата'].max()]['Заражений'].iloc[0]
    p.text = f'Заболеваний: {int(confirmed)}'
    p.font.size = pptx.util.Pt(5)
    p.font.bold = True
    p.font.name = 'Trebuchet MS'


    # Second graphic (deaths)
    fig = px.line(country_df.sort_values('Дата'), x="Дата", y="Смертей за день", color_discrete_sequence=["blue"], template='plotly_white')
    fig.update_layout(
        margin=dict(t=0,l=0,b=0,r=15)
    )
    fig.update_yaxes(title_text="")
    fig.update_xaxes(title_text="")
    pic_path = f'{today}/graphics/Смертей {country}.jpeg'
    fig.write_image(pic_path)

    slide, x, y = pic_positions.pop(0)
    prs.slides[slide].shapes.add_picture(pic_path, pptx.util.Cm(x), pptx.util.Cm(y),
                               width=pptx.util.Cm(pic_width), height=pptx.util.Cm(pic_height))


    # Deaths text
    x, y = deaths_text_positions.pop(0)
    txBox = prs.slides[slide].shapes.add_textbox(pptx.util.Cm(x), pptx.util.Cm(y),
                               width=pptx.util.Cm(1), height=pptx.util.Cm(1))
    tf = txBox.text_frame
    p = tf.paragraphs[0]
    deaths = country_df[country_df['Дата'] == country_df['Дата'].max()]['Смертей'].iloc[0]
    p.text = f'Летальных исходов: {int(deaths)} ({round(deaths/confirmed * 100, 1)}%)'
    p.font.size = pptx.util.Pt(5)
    p.font.bold = True
    p.font.name = 'Trebuchet MS'


    #vaccine_graphics
    print(country)
    vac_country = vac[vac['location'] == country]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=vac_country['date'], y=vac_country['people_fully_vaccinated'],
                        mode='lines',
                        name='Итого привито 2 комп., чел.', connectgaps=True))
    fig.add_trace(go.Scatter(x=vac_country['date'], y=vac_country['people_vaccinated'],
                        mode='lines',
                        name='Итого привито 1 комп., чел.', connectgaps=True))
    fig.add_trace(go.Scatter(x=vac_country['date'], y=vac_country['total_vaccinations'],
                        mode='lines', name='Итого прививок, шт.', connectgaps=True))
    fig.update_layout(
        margin=dict(t=0,l=0,b=0,r=15)
    )
    fig.update_layout(template='plotly_white')
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=0.5
    ))
    pic_path = f'{today}/graphics/Вакцина {country}.jpeg'
    fig.write_image(pic_path)
    slide, x, y = pic_positions.pop(0)
    prs.slides[slide].shapes.add_picture(pic_path, pptx.util.Cm(x), pptx.util.Cm(y),
                               width=pptx.util.Cm(pic_width), height=pptx.util.Cm(pic_height))
    

    # vaccine info
    last_date = vac_country[vac_country['date'] == vac_country['date'].max()]
    people_fully_vaccinated = last_date['people_fully_vaccinated'].iloc[0]
    people_fully_vaccinated_per_hundred = last_date['people_fully_vaccinated_per_hundred'].iloc[0]
    people_vaccinated = last_date['people_vaccinated'].iloc[0]
    people_vaccinated_per_hundred = last_date['people_vaccinated_per_hundred'].iloc[0]
    daily_vaccinations = last_date['daily_vaccinations'].iloc[0]
    daily_vaccinations_per_hundred = last_date['daily_vaccinations_per_million'].iloc[0]/100000


    x, y = vaccine_text_positions.pop(0)
    txBox = prs.slides[slide].shapes.add_textbox(pptx.util.Cm(x), pptx.util.Cm(y),
                               width=pptx.util.Cm(1), height=pptx.util.Cm(1))
    tf = txBox.text_frame
    p = tf.paragraphs[0]
    p.text = f'Привито {people_fully_vaccinated_per_hundred}% ({int(people_fully_vaccinated)} чел.)'
    p.font.size = pptx.util.Pt(5)
    p.font.name = 'Trebuchet MS'

    p = tf.add_paragraph()
    p.text = f'Привито 1 компонентом {people_vaccinated_per_hundred}% ({int(people_vaccinated)} чел.)'
    p.font.size = pptx.util.Pt(5)
    p.font.name = 'Trebuchet MS'

    p = tf.add_paragraph()
    p.text = f'Привито за день {int(daily_vaccinations)} чел. ({daily_vaccinations_per_hundred}% от населения)'
    p.font.size = pptx.util.Pt(5)
    p.font.name = 'Trebuchet MS'


    # table in 1 slide
    table_row = [country,
                 str(int(country_df[country_df['Дата'] == country_df['Дата'].max()]['Заражений(МА)'].iloc[0])),
                 str(int(country_df[country_df['Дата'] == country_df['Дата'].max()]['Заражений за день'].iloc[0])),
                 str(people_vaccinated_per_hundred),
                 str(people_fully_vaccinated_per_hundred),
                 str(int(country_df[country_df['Дата'] == country_df['Дата'].max()]['Смертей за день'].iloc[0]))]
    for j, elem in enumerate(table_row):
        prs.slides[0].shapes[7].table.cell(i+1, j).text = elem
        p = prs.slides[0].shapes[7].table.cell(i+1, j).text_frame.paragraphs[0]
        p.font.size = pptx.util.Pt(11)
        p.alignment = PP_ALIGN.CENTER

    
prs.save(rename_input_filename('out.pptx'))

In [ ]:
# files.download('/content/29-07-2021/out_29-07-2021.pptx')

#Результат

In [ ]:
def main(world_input_filename: str, russia_input_filename: str):
    russia(russia_input_filename)
    world(world_input_filename)
    report_docx()


In [ ]:
main(world_input_filename, russia_input_filename)

In [ ]:
!zip -r archive.zip $FOLDER_NAME

In [ ]:
print(f'Время работы скрипта: {(time.time() - start_time)/60} минут')

In [ ]:
files.download('archive.zip')